## Gender Classification Using CNN ##

## machine learning model processing 
* Problem Definition -> Classification of Gender
* Data Preprocessing -> Prepare Features and Labels, Data Scaling
* Model Selection -> Logistic Regression, SVM, Random Forest
* Model Evaluation -> 1. Evaluate on test or validation data, 2. Metrics (Accuracy, Precision, F1-Score)
* Model Training -> Fit the model to the train data
* Model Visualization -> Display the Confusion Matrix
* Hyperparameter Tuning -> Grid Search, Randomised Search
* Model Validation ->Validate model generalization on unseen data
* Model Save -> Keras

**Problem Definition::**

*Objective: Classify gender (Male/Female) based on input features (e.g., facial measurements, extracted image features).
Type: Binary Classification*

In [120]:
# Importing Necessary Libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [121]:
# Data Preprocessing

# Define paths
train_path = 'dataset/train'
val_path = 'dataset/val'

# Image dimensions
img_height, img_width = 128, 128

# Data generators with normalization
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess images
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important for consistent predictions
)


Found 1926 images belonging to 2 classes.
Found 422 images belonging to 2 classes.


In [122]:
# Model Selection
def shallow_cnn():  # Model 1
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

def medium_cnn():  # Model 2
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

def deep_vgg_like_cnn():  # Model 3
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
# Train the Models
models = {
    "Shallow CNN": shallow_cnn(),
    "Medium CNN": medium_cnn(),
    "Deep VGG-like CNN": deep_vgg_like_cnn()
}

histories = {}
results = {}

for name, model in models.items():
    print(f"\n🔁 Training: {name}")
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=val_generator,
        verbose=1
    )

    loss, acc = model.evaluate(val_generator, verbose=0)
    print(f"✅ {name} - Test Accuracy: {acc:.4f}")
    
    histories[name] = history
    results[name] = (model, acc) 


🔁 Training: Shallow CNN
Epoch 1/10
45/61 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.7789 - loss: 2.6636

In [ ]:
#Fit the best model
def shallow_cnn(learning_rate):  # <== Accept learning_rate here
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
    
# Fit the model (if not already trained)
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    verbose=1
)


In [ ]:
#Model Validation
loss, acc = model.evaluate(val_generator, verbose=0)
print(f"✅ Final Test Accuracy: {acc:.4f}")

In [ ]:
# Get true labels and predicted probabilities
y_true = val_generator.classes
y_pred_prob = model.predict(val_generator)
y_pred = (y_pred_prob > 0.5).astype("int32").reshape(-1)

In [ ]:
print("✅ Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Male', 'Female']))

In [ ]:
# Visualization
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Male', 'Female'], yticklabels=['Male', 'Female'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Data loader function
def get_data(batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary'
    )

    val_gen = val_datagen.flow_from_directory(
        val_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )
    return train_gen, val_gen


In [ ]:
# Hyperparameter tuning loop
best_acc = 0
best_config = None
best_model = None

for lr in learning_rates:
    for batch in batch_sizes:
        print(f"\n🔧 Training with LR={lr}, Batch Size={batch}")
        
        train_gen, val_gen = get_data(batch)
        model = shallow_cnn(lr)
        
        model.fit(train_gen, epochs=10, validation_data=val_gen, verbose=0)
        loss, acc = model.evaluate(val_gen, verbose=0)
        
        print(f"✅ Accuracy: {acc:.4f}")
        
        if acc > best_acc:
            best_acc = acc
            best_config = (lr, batch)
            best_model = model

print(f"\n🏆 Best Accuracy: {best_acc:.4f} with LR={best_config[0]}, Batch Size={best_config[1]}")

In [ ]:
from tensorflow.keras.models import load_model

# Load the model (from .keras file)
model.save("final_best_gender_model.h5")
print("✅ Best tuned model loaded from: final_best_gender_model.h5")
